# Acoustic seismic wave propagation with Serendipyty

The purpose of this Jupyter notebook is to show how to set up and perform a simple acoustic wave propagation simulation using the Python package [Serendipyty](https://github.com/serendipyty/serendipyty). First, various standard Python packages are imported and their settings specified. Then, the temporal and spatial parameters for the simulation are definied. These parameters are then used to definte the inpute and out parameters and to create the models for the material properties (velocity and density for an acoustic medium). Finally, the simulation is created and the wavefields are displayed and animated. 

## Setup: matplotlib

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.animation as animation
from IPython.display import HTML

mpl.rc('image', interpolation='none')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.constrained_layout.use'] = True

# Get current size
fig_size = plt.rcParams["figure.figsize"]
print("The current size of a figure is {}".format(fig_size))

## Setup: Import packages and set configuration parameters

In [ ]:
# System
import sys
from timeit import default_timer as timer

# HPC
from joblib import Parallel, delayed
import multiprocessing

# Numerical
import numpy as np

# Set test to True for development phase
test = True
if test:
    %load_ext autoreload
    %autoreload 2
    sys.dont_write_bytecode = True

# Serendipyty
from serendipyty.seismic.modelling import awe2d
from serendipyty.seismic.modelling.bcs import PmlBc

from serendipyty.seismic.utils.fd import stability
from serendipyty.seismic.utils.fd import dispersion
from serendipyty.seismic.utils.util import rectangle
from serendipyty.seismic.utils.util import oneface

from serendipyty.seismic.utils.dispersion import itdt3
from serendipyty.seismic.input.wavelets import RickerWavelet
from serendipyty.seismic.input.sources import PointSource
from serendipyty.seismic.model.models import AcousticModel
from serendipyty.seismic.hpc.hpc import BaseHpc

# Pandas
import pandas as pd

# Set numerical precision
DTYPE = np.float64

## Spatial parameters

In [ ]:
## Domain properties
# Extent in # of cells
nx = 201
nz = 101
# Sampling in m
dx = DTYPE(5)
dz = DTYPE(5)

## Source
# Source locations in m
src_loc_m = np.array([600, 0, 250], dtype=DTYPE)
# Source locations in grid point
src_loc = np.array(src_loc_m/dx, dtype=np.uint)
# Type: 'q' for monopole source, 'fx' or 'fz' for dipole source
sourcetype = 'q'
# Central frequency
fc = DTYPE(30)

## Absorbing boundary conditions
# Size of absorbing boundary
npml = 30

# Create Perfectly Matched Layers (PMLs) bondary conditions class
bc = PmlBc(npml=npml, freesurface=True)

## Temporal parameters

In [ ]:
# Duration of simulation in s
tmax = 5.00
# Sampling in s
dt = DTYPE(0.001)
# Number of time samples
nt = int(tmax/dt)+1

## Create spatial and temporal axes

In [ ]:
x = np.arange(nx)*dx
z = np.arange(nz)*dz
t = np.arange(nt)*dt

## Material parameters: simple layered model

In [ ]:
# Velocity and density of first layer
vp0 = 2000
rho0 = 1000

# Velocity and density ndarrays
vp = np.ones((nx, nz), dtype=DTYPE)*vp0
rho = np.ones((nx, nz), dtype=DTYPE)*rho0

# Create second layer
dip = 0.1
inta = (-dip)*x + 150.0
mask = np.zeros((nx,nz), dtype=np.bool)
for i in range(nx):
    mask[i, np.rint(inta[i]/dx).astype(int):] = True
vp[mask] = 2500
rho[mask] = 1500

# Create third layer
intb = 350
vp[:,np.rint(intb/dx).astype(int):] = 2300
rho[:,np.rint(intb/dx).astype(int):] = 1300

# Create model class
model = AcousticModel(dx, vp=vp, rho=rho)

# Plot material parameters
model.plot(extent=(x, z), colorbar=True, figsize=(15,3))

## Dispersion and stability analysis

The Courant number is used to restrict the time-step in explicit time-marching computer simulations. For example, if a wave is crossing a discrete grid distance ($\Delta x$), then the time-step must be less than the time needed for the wave to travel to an adjacent grid point, otherwise the simulation will produce incorrect results. For 4th order spatial derivatives, the Courant number is $0.606$ and for stability the time discretization ($\Delta t$) must satisfy

$$\Delta t \leq \frac{0.606 \Delta x}{v_{max}},$$

where $v_{max}$ is the highest velocity of the medium.

Besides unstable solutions, wavefield dispersion also occurs because finite-difference schemes are intrinsically dispersive. The widespread rule of thumb *5 points per wavelength* is used to avoid dispersion. The grid discretization ($\Delta x$) must satisfy

$$\Delta x \le \frac{v_{vim}}{5 f_{max}},$$

where $v_{min}$ is the lowest velocity of the medium and $f_{max}$ the maximum frequency of the source wavelet.

In [ ]:
# Compute dispersion criterion
dx_no_dispersion = dispersion(vp.min(), dx, fc, coeff=2.0)
print('To avoid (strong) spatial dispersion, the spatial sampling dx should be < {:.6}m'.format(dx_no_dispersion))

# Compute the stability criterion
dt_stable = stability(vp.max(), dx, dt)
print('To avoid an unstable simulation, the temporal sampling dt should be < {:.6}s'.format(dt_stable))

# Verify that the criteria are not violated
if dx < dx_no_dispersion and dt < dt_stable:
    print('Woot woot! The sampling values dx={:.6}m and dt={:.6}s are good for this simulation!'.format(dx, dt))

## Source

The expression of the Ricker wavelet is $w(t) = (1 - 2 \pi^2 f_c^2 t^2)exp(-\pi^2 f_c^2 t^2)$, where $f_c$ is the central (peak) frequency.

In [ ]:
# Choose the wavelet
wav = RickerWavelet(t, fc=fc, delay=0.05)

# Choose the source type
src = PointSource(src_loc, wav, 'q')

# Plot the source wavelet
line = src.plot(tmax=0.5)

## Receivers along two horizontal surfaces

In [ ]:
# Number of disjoint subdomains
# Currently, it is possible to use only one domain
# Extension to more domains is work in progress
nsub = 1

# Initialize arrays
semt_origins = np.ndarray((nsub, 4), dtype=np.uint)
srec_origins = np.ndarray((nsub, 4), dtype=np.uint)

# order: xemt zemt nxemt nzemt
semt_origins[0,:] = (0,  35,  nx,  nz-35)

# number of gridpoints between recording and emitting surfaces
ngpts = 2

# order: xrec zrec nxrec nzrec
srec_origins = semt_origins + np.array((0, ngpts, 0, 0), dtype=np.uint)

# Locations
# TO DO: this currently works fine because nsub is == 1
# When nsub > 1, I need to concatenate the locations along axis=0
# WARNING: the function oneface uses generator expressions
# to create the numpy array of the locations.
for i in range(nsub):
    semt_locs = oneface(faces=(2,),
                        origin=(semt_origins[i][0], 0, semt_origins[i][1]),
                        number_of_cells=(semt_origins[i][2], 0, semt_origins[i][3]),
                        cell_size=(1,1,1)
                       )

for i in range(nsub):
    srec_locs = oneface(faces=(2,),
                        origin=(srec_origins[i][0], 0, srec_origins[i][1]),
                        number_of_cells=(srec_origins[i][2], 0, srec_origins[i][3]),
                        cell_size=(1,1,1)
                       )

In [ ]:
# Create a Pandas DataFrame from the srec_locs array
# This could be useful in the future
df_srec_locs = pd.DataFrame(srec_locs)
df_srec_locs.columns = ['x', 'y', 'z', 'face']
df_srec_locs.head()

## Define outputs

In [ ]:
# Outputs are not implemented as a class yet.
# We still need to finalize some of the output format
# such as sub_volume_boundary.
outparam = []
# Wavefield snapshots
outparam.append({'type': 'slice',
                 'timestep_increment': 10
                })
# Pressure wavefield along the semt_locs
outparam.append({'type': 'sub_volume_boundary',
                'attribute': 'p',
                'receiver_locations': semt_locs,
                'stagger_on_sub_volume': True,
                'timestep_increment': 10
                })
# Pressure wavefield along the srec_locs
outparam.append({'type': 'sub_volume_boundary',
                'attribute': 'vn',
                'receiver_locations': srec_locs,
                'stagger_on_sub_volume': True,
                'timestep_increment': 10
                })

## HPC parameters

In [ ]:
ncores = multiprocessing.cpu_count()
print('This computer has {} cores'.format(ncores))

In [ ]:
# HPC class
hpc = BaseHpc(omp_num_threads=3)

## Run simulation

Finite-difference time-domain solution of the 2D acoustic wave equation implemented using [Cython](http://cython.org/) and [typed memoryviews](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html).

The acoustic forward solver of Serendipyty solves the 2D wave equation by approximating the derivatives
in the wave equation by finite-differences. The acoustic wave equation is defined through the first-order linearized
systems of Newton’s and Hooke’s law:

$$\frac{\partial v_x}{\partial t} = -\frac{1}{\rho}\frac{\partial p}{\partial x}$$
$$\frac{\partial v_z}{\partial t} = -\frac{1}{\rho}\frac{\partial p}{\partial z}$$
$$\frac{\partial p}{\partial t} = -v^2\rho\{\frac{\partial v_x}{\partial x}+\frac{\partial v_z}{\partial z}\}$$

where $v_x$ and $v_z$ are the particle velocity components in the $x$ and $z$-direction, respectively, $p$ is the acoustic pressure, $\rho$ is the mass density of the medium and $v$ the velocity. The implemented Finite-Difference codes make use of a staggered grid and is following the grid layout as described in [Virieux (1986)](http://geodynamics.usc.edu/~becker/teaching/557/reading/Virieux1987.pdf).


In [ ]:
outputs = awe2d.forward(model,src,outparam,bc,hpc)

### List outputs

In [ ]:
print('The outputs are:')
for out in outputs.keys():
    print('- {}'.format(out))

if 'sub_volume_boundary' in outputs.keys():
    print('There are {} sub_volume_boundary outputs'.format(outputs['sub_volume_boundary'].shape[0]))

## Plot wave field snapshot

In [ ]:
# Add model in the background
alpha = model.model[..., 0]
slicealpha = outputs['slice'][0, ...] + (alpha[None, ...] - alpha.mean())/1

In [ ]:
# Prepare figure

# Figure
figsize=(12, 5)
ndim = 2
fig, axs = plt.subplots(nrows=1, ncols=ndim, figsize=figsize, facecolor='w', edgecolor='k', squeeze=True)

# Plot options
clip=5e3
extent=[0, x[-1], z[-1], 0]
aspect='equal'
cmap='seismic'
plotopts0 = {
    'vmin': -clip,
    'vmax': +clip,
    'aspect': aspect,
    'cmap': cmap,
    'animated': True,
    'interpolation': 'bilinear',
    'extent': extent
}
clip=1e3
extent=[0, x[-1], t[1000], 0]
aspect=5e2
cmap='seismic'
plotopts1 = {
    'vmin': -clip,
    'vmax': +clip,
    'aspect': aspect,
    'cmap': cmap,
    'animated': True,
    'interpolation': 'bilinear',
    'extent': extent
}
i = 20
im0 = axs[0].imshow(slicealpha[i, ...].T, **plotopts0)
axs[0].scatter(srec_locs[5::10, 0]*dx, srec_locs[5::10, 2]*dz, color='k')
im1 = axs[1].imshow(outputs['sub_volume_boundary'][0, :100, :], **plotopts1)

axs[0].yaxis.set_label_text('Depth [m]')
axs[1].yaxis.set_label_text('Time [s]')

# ax0.set_aspect('equal')
axs[0].set_title('Pressure')
axs[1].set_title('Normal velocity')

for ax in axs:
    # Remove the ugly ticks
    ax.tick_params(
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,        # ticks along the top edge are off
        left=False,        # ticks along the top edge are off
        right=False        # ticks along the top edge are off
    )
    ax.xaxis.set_label_text('Horizontal location [m]')

## Create wavefield animation

In [ ]:
blit = True
interval = 50
nframes = 100

def init():
    im0.set_array(slicealpha[0, ...].T)
    axs[0].scatter(srec_locs[5::10, 0]*dx, srec_locs[5::10, 2]*dz, color='k')
    im1.set_array(np.zeros_like(outputs['sub_volume_boundary'][0,  :100, :]))
    return im0, im1,

def snap(i, snap, vb):
    im0.set_array(snap[i, ...].T)
    temp = im1.get_array()
    # print(temp.shape)
    temp[i, ...] = vb[i, ...]
    im1.set_array(temp)
    # Set title
    title0 = axs[0].title.set_text('Pressure (t:{0:.4})'.format(i*dt*10))
    return im0, im1,

anim = animation.FuncAnimation(fig, snap, init_func=init,
                              fargs=(slicealpha, outputs['sub_volume_boundary'][0,  :100, :]), frames=nframes,
                              blit=blit, interval=interval, repeat=True)

In [ ]:
HTML(anim.to_html5_video())
# HTML(anim.to_jshtml())